<a href="https://colab.research.google.com/github/radhakrishnan-omotec/arwan-iris-dog-repo/blob/main/Project9_Dog_Breed_Classification_Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
kabilan03_dogbreedclassification_path = kagglehub.dataset_download('kabilan03/dogbreedclassification')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# <center> Dog Breed Classification using Inception

**Team Members:**
* **ADHITHAN P**
* **ANUVARSHINI S P**
* **KABILAN N**
* **SIVAMARAN M A C**

**Dataset** -> https://www.kaggle.com/kabilan03/dogbreedclassification

## Importing Packages

In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import cv2
import matplotlib
from matplotlib import pyplot
from matplotlib import pyplot as plt
from tensorflow import keras
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from sklearn.model_selection import train_test_split

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2

from tensorflow import keras
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers import Flatten
from matplotlib import pyplot
from keras import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score
import seaborn as sns
from keras.layers import Dense, Dropout, Activation, Flatten,BatchNormalization
import numpy as np
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt

from keras import backend as K
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications import InceptionV3

### Loading Dataset

In [ ]:
def load_subdata(data):
    X1 = []
    y1 = []
    size = 229
    lbl = 0
    dic = {}
    for folder_name in os.listdir(data):

        Label = lbl
        dic[folder_name] = Label
        count_imag = 0;

        for filename in os.listdir(data + '/' + folder_name):
            image = tf.keras.preprocessing.image.load_img(data +'/'+ folder_name + '/' + filename,color_mode='rgb', target_size=(size,size))
            arr = keras.preprocessing.image.img_to_array(image)
            X1.append(np.array([arr]))
            y1.append(Label)
        lbl +=1

    X1 = np.asarray(X1).reshape(len(X1),229,229,3).astype(np.uint8)
    y1 = np.asarray(y1).astype(np.uint8)
    return X1,y1, dic

In [ ]:
X_train, y_train, label_dic = load_subdata(r'../input/dogbreedclassification/Dog Breed Classification/train')
X_test, y_test, test_label_dic = load_subdata(r'../input/dogbreedclassification/Dog Breed Classification/test')
X_val, y_val, val_label_dic = load_subdata(r'../input/dogbreedclassification/Dog Breed Classification/val')

In [ ]:
print("Training Images Shape (x train shape) :", X_train.shape)
print("Label of training images (y train shape) :",y_train.shape)

print("Validation Images Shape (x val shape) :",X_val.shape)
print("Label of Validation images (y val shape) :",y_val.shape)

print("Test Images Shape (x val shape) :",X_test.shape)
print("Label of Test images (y val shape) :",y_test.shape)

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
label_count = dict(zip(unique, counts))

### One Hot Encoding

In [ ]:
y_trainHot = np.uint8(to_categorical(y_train, num_classes = 93))
y_testHot = np.uint8(to_categorical(y_test, num_classes = 93))
y_valHot = np.uint8(to_categorical(y_val, num_classes = 93))

### Shuffling Dataset

In [ ]:
def shuffle(a, b):
    shuffled_a = np.empty(a.shape, dtype=np.uint8)
    shuffled_b = np.empty(b.shape, dtype=np.uint8)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [ ]:
X_train_s, y_train_s = shuffle(X_train, y_trainHot)
X_test_s, y_test_s = shuffle(X_test, y_testHot)
X_val_s, y_val_s = shuffle(X_val, y_valHot)

In [ ]:
print("One hot encoded labels")
print(y_train_s.shape)
print(y_test_s.shape)
print(y_val_s.shape)

In [ ]:
plt.figure(figsize=(10,10))
plt.title('Random Image from Dataset')
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_val_s[i])

plt.show()

## Inception Model

In [ ]:
def Inception_Net():
    base_model =InceptionV3(input_shape = (229, 229, 3), # Shape of our images
                                 include_top = False,
                                 weights = 'imagenet',pooling='avg',classes=93)
    base_model.trainable = True
    base_model.layers[0].trainable=False
    #pool_layer=GlobalAveragePooling2D()
    prediction_layer = Dense(units=93, activation='softmax')
    model = Sequential()
    model.add(base_model)
    model.add(Dropout(0.4))
    #model.add(pool_layer)

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(units=1024, activation='relu'))
    model.add(prediction_layer)
    return model


In [ ]:
incp_model=Inception_Net()

In [ ]:
incp_model.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(learning_rate=0.0001),
              metrics = ['accuracy']) #when the learning rate was set to 0.01, the model showed a poor performance comparated to lr=0.0001
incp_model.summary()

In [ ]:
hist =incp_model.fit(X_train_s, y_train_s, epochs=10,batch_size=128,validation_data=(X_val_s,y_val_s))

In [ ]:
def plot_model(history):
    fig,(ax1,ax2)=plt.subplots(1,2,figsize=(12,4))
    fig.suptitle('Model Accuracy and Loss')

    ax1.plot(hist.history['accuracy'])
    ax1.plot(hist.history['val_accuracy'])
    ax1.title.set_text('Accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train','Valid'],loc=4)

    ax2.plot(hist.history['loss'])
    ax2.plot(hist.history['val_loss'])
    ax2.title.set_text('Loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train','Valid'],loc=1)

    fig.show()

plot_model(hist)

In [ ]:
def plot_cf_mat(cf_matrix):

    classes = list(label_dic.keys())
    plt.rcParams["figure.figsize"] = (10,7)
    ax = plt.axes()
    sns.heatmap(cf_matrix, annot=True,
              annot_kws={"size": 10},
              xticklabels=classes,
              yticklabels=classes, ax = ax,fmt="d")
    ax.set_title('Confusion matrix')
    ax.set_xlabel("Predicted")
    ax.set_ylabel("Actual")
    plt.show()

In [ ]:
def model_Evaluate(model):

    # Predict values for Test dataset
    y_pred = model.predict(X_test_s)
    y_test = y_test_s
    Y_pred = np.argmax(y_pred, axis=1)
    Y_Test = np.argmax(y_test, axis=1)

    class_names = list(label_dic.keys())
    print("Classification Report:")

    # Print the evaluation metrics for the dataset.
    print(classification_report(Y_Test, Y_pred, target_names=class_names))

    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(Y_Test, Y_pred)

    accuracy = recall_score(Y_Test, Y_pred, average=None)
    precision=precision_score(Y_Test,Y_pred,average=None)

    #class_names = class_names
    class_names = list(label_dic.keys())

    print("Confusion Matrix:")

    print(cf_matrix)
    plot_cf_mat(cf_matrix)
    ## Display the visualization of the Confusion Matrix.
    plt.show()

    print("Accuracy for each class: ")

    for i in range(len(accuracy)):
        print(f"{class_names[i]}:  {format(accuracy[i]*100, '.2f')}%")
    return accuracy,precision

In [ ]:
accuracy_incp, precision_incp = model_Evaluate(incp_model)

## Saving Model

In [ ]:
Inception_Model = incp_model.to_json()
with open("incp_model.json", "w") as json_file:
    json_file.write(Inception_Model)
# serialize weights to HDF5
incp_model.save_weights("incp_model.h5")
print("Saved model to disk")

## Load Model

In [ ]:
from keras.models import model_from_json
json_file = open('incp_model.json', 'r')
Inception_Model_json = json_file.read()
json_file.close()
Inception_Model = model_from_json(Inception_Model_json)
# load weights into new model
Inception_Model.load_weights("incp_model.h5")
print("Loaded model from disk")

In [ ]:
print("Printing layers of Model :")
for layer in Inception_Model.layers:
    print(layer)

print()
print("Feature extraction from the model:")
feature_layer = Inception_Model.get_layer('inception_v3')
print(feature_layer)

In [ ]:
incep_feature_model = keras.Model(inputs = feature_layer.inputs, outputs = feature_layer.outputs)


# Showing Inception Model Architecture
# print("Inception Model Architecture")
# keras.utils.plot_model(feature_model,'Inception.png',show_shapes=True, show_layer_names=False, rankdir='TB', expand_nested=True, dpi=75)
# rankdir='TB' -> top to bottom
#feature_model.summary()

In [ ]:
print("Inception Model Architecture")
keras.utils.plot_model(incep_feature_model,'Inception_Model.png',show_shapes=True, show_layer_names=False, rankdir='TB', expand_nested=True, dpi=75)

## ML classifiers

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [ ]:
Y_Train = np.argmax(y_train_s,axis=1)
Y_Test = np.argmax(y_test_s,axis=1)

In [ ]:
train_features = incep_feature_model.predict(X_train_s)
test_features = incep_feature_model.predict(X_test_s)

print("Training data size:", train_features.shape)
print("Test data size:", test_features.shape)

In [ ]:
def classifier(classifier_):
    class_names = list(label_dic.keys())
    print(f'Classifier: {str(classifier_)}')
    classifier_.fit(train_features, Y_Train)
    Y_pred = classifier_.predict(test_features)
    acc = accuracy_score(Y_Test,Y_pred)
    print(f"Accuracy: {acc}")
    print(classification_report(Y_Test,Y_pred))
    accuracy = recall_score(Y_Test, Y_pred, average=None)
    for i in range(len(accuracy)):
        print(f"{class_names[i]}:  {format(accuracy[i]*100, '.2f')}%")
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(Y_Test, Y_pred)
    print(cf_matrix)
    plot_cf_mat(cf_matrix)
    ## Display the visualization of the Confusion Matrix.
    plt.show()

In [ ]:
classifier(DecisionTreeClassifier())

In [ ]:
classifier(RandomForestClassifier())

In [ ]:
# classifier(GradientBoostingClassifier())

In [ ]:
classifier(SVC(kernel='linear'))

In [ ]:
classifier(GaussianNB())

In [ ]:
classifier(KNeighborsClassifier())

In [ ]:
#classifier(LGBMClassifier())